In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import matplotlib.pyplot as plt
import warnings
import seaborn as sns
from sklearn.metrics import mean_squared_error
from numba import jit, cuda
warnings.filterwarnings("ignore")
import os
import joblib
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
#x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)


models=[]

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

df = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/train.csv')
x_test = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv')

In [2]:
df_test = x_test.copy()
x=df.drop(['target'],axis=1)
y=df[['target']]

In [3]:
def data_preprocessing(x):
    x['imbalance_ratio'] = x['imbalance_size'] / x['matched_size']
    x['bid_ask_volume_diff'] = x['ask_size'] - x['bid_size']
    x['mid_price'] = (x['ask_price'] + x['bid_price']) / 2
    x['bid_plus_ask_sizes'] = x['bid_size'] + x['ask_size']
    if 'time_id' not in x.columns:
        x['time_id']=[0]*x.shape[0]
    x.drop(['far_price','near_price','imbalance_size','matched_size','ask_size','bid_size','row_id','time_id'],axis=1,inplace=True)
    for i in x.columns:
        if x[i].isnull().sum()>0:
            x[i].fillna(x[i].mean(),inplace=True)
    return x

In [4]:
x=data_preprocessing(x)
x_test = data_preprocessing(x_test)
y.fillna(y.mean(),inplace=True)

In [5]:
from xgboost import XGBRegressor
XG=XGBRegressor()
XG.fit(x,y)

#pred_XG_test=XG.predict(x_test)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [6]:
def get_preds(data):
    data = data.fillna(0)
    data = data_preprocessing(data)
    values = XG.predict(data)
    return values

In [7]:
# data = {"time_id":df_test["time_id"],"row_id":df_test["row_id"],"target":pred_XG_test}
# final_preds = pd.DataFrame(data,columns=["time_id","row_id","target"])

In [8]:
import optiver2023
env = optiver2023.make_env()
iter_test = env.iter_test()

In [9]:
counter = 0
for (test, revealed_targets, sample_prediction) in iter_test:
    if counter == 0:
        print(test.head(3))
        print(revealed_targets.head(3))
        print(sample_prediction.head(3))
    sample_prediction['target'] = get_preds(test)
    env.predict(sample_prediction)
    sample_prediction = sample_prediction.fillna(0)
    sample_prediction = 10000
    counter += 1

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
   stock_id  date_id  seconds_in_bucket  imbalance_size  \
0         0      478                  0      3753451.43   
1         1      478                  0       985977.11   
2         2      478                  0       599128.74   

   imbalance_buy_sell_flag  reference_price  matched_size  far_price  \
0                       -1         0.999875   11548975.43        NaN   
1                       -1         1.000245    3850033.97        NaN   
2                        1         1.000584    4359198.25        NaN   

   near_price  bid_price  bid_size  ask_price  ask_size  wap   row_id  
0         NaN   0.999875  22940.00   1.000050   9177.60  1.0  478_0_0  
1         NaN   0.999940   1967.90   1.000601  19692.00  1.0  478_0_1  
2         NaN   0.999918   4488.22   1.000636  34955.12  1.0  478_0_2  
   stock_id  date_id  seconds_in_bucket  revealed_target  rev